In [1]:
%%capture
import tensorflow as tf
from tensorflow import keras
from scipy.io import loadmat
from tensorflow.keras import regularizers
import os
import time
import numpy as np
import gc
from tensorflow.keras.models import model_from_json
from tensorflow.keras.models import load_model
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
from tensorflow.keras import backend as K
from sklearn.metrics import r2_score
from sklearn import preprocessing
import gc
import h5py
import math
from IPython.display import HTML, display
import tabulate

In [2]:
def scale(x):
    return (x -np.mean(x)) / np.std(x)

In [3]:
def preprocess(x): 
    return x[:,:,:int(0.75*x.shape[2])]

In [4]:
os.chdir("Saved Models")

##### Load Model for heat1
json_file = open('model_heat1.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model_1 = model_from_json(loaded_model_json)

# load weights into new model
loaded_model_1.load_weights("model_heat1.h5")
print("Loaded model from disk")

# evaluate loaded model on test data 
# Define X_test & Y_test data first
loaded_model_1.compile(optimizer = tf.train.AdamOptimizer(), loss = "mse")

#### Load Model for heat2
json_file = open('model_heat2.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model_2 = model_from_json(loaded_model_json)

# load weights into new model
loaded_model_2.load_weights("model_heat2.h5")
print("Loaded model from disk")

# evaluate loaded model on test data 
# Define X_test & Y_test data first
loaded_model_2.compile(optimizer = tf.train.AdamOptimizer(), loss = "mse")

#### Load Model for heat3
json_file = open('model_heat3.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model_3 = model_from_json(loaded_model_json)

# load weights into new model
loaded_model_3.load_weights("model_heat3.h5")
print("Loaded model from disk")

# evaluate loaded model on test data 
# Define X_test & Y_test data first
loaded_model_3.compile(optimizer = tf.train.AdamOptimizer(), loss = "mse")

Loaded model from disk
Loaded model from disk
Loaded model from disk


In [5]:
out = [["Data Source", "Result", "Heat1", "Heat2", "Heat3"]]
coarseArray = [4, 16]
for coarse in coarseArray:
    dirName = "/home/llin2948/dscaptstone/Re15sh20_DNScoarse" + str(coarse)
    os.chdir(dirName)
    u = preprocess(scale(loadmat('u_F_xyz_T1_DNS'+str(coarse)+'.mat')["u_F"]))
    v = preprocess(scale(loadmat('v_F_xyz_T1_DNS'+str(coarse)+'.mat')["v_F"]))
    w = preprocess(scale(loadmat('w_F_xyz_T1_DNS'+str(coarse)+'.mat')["w_F"]))
    heat1 = preprocess(scale(loadmat('Heat1_xyz_T1_DNS'+str(coarse)+'.mat')["Heat1"]))
    heat2 = preprocess(scale(loadmat('Heat2_xyz_T1_DNS'+str(coarse)+'.mat')["Heat2"]))
    heat3 = preprocess(scale(loadmat('Heat3_xyz_T1_DNS'+str(coarse)+'.mat')["Heat3"]))
    #tke = preprocess(scale(loadmat('TKE_F_xyz_T1_DNS'+str(coarse)+'.mat')["TKE_F"]))

    x = np.array([u, v, w])
    x = np.transpose(x, [1, 2, 3, 0])
    x = np.pad(x, ((3,3), (3,3), (3,3), (0,0)), 'constant', constant_values = 0)

    size = 3
    y_pred_12 = np.array([])
    y_pred_13 = np.array([])
    y_pred_23 = np.array([])
    y_true_12 = np.array([])
    y_true_13 = np.array([])
    y_true_23 = np.array([])
    u_idx = 0
    v_idx = 0
    z_idx = 0
    for u_idx in range(0, math.ceil(x.shape[0]/151)):
        for v_idx in range(0, math.ceil(x.shape[1]/101)):
            for z_idx in range(0, math.ceil(x.shape[2]/60)):
                u_lowerbound = u_idx*151+size
                u_upperbound = x.shape[0] - size if (u_idx+1)*151 >= x.shape[0] else (u_idx+1)*151 - size
                v_lowerbound = v_idx*101+size
                v_upperbound = x.shape[1] - size if (v_idx+1)*101 >= x.shape[1] else (v_idx+1)*101 - size
                z_lowerbound = z_idx*60+size
                z_upperbound = x.shape[2] - size if (z_idx+1)*60 >= x.shape[2] else (z_idx+1)*60 - size
                print(str(u_lowerbound) + ' ' + str(u_upperbound))
                print(str(v_lowerbound) + ' ' + str(v_upperbound))
                print(str(z_lowerbound) + ' ' + str(z_upperbound))
                sample = []
                y_heat1 = []
                y_heat2 = []
                y_heat3 = []
                for i in tqdm_notebook(range(u_lowerbound, u_upperbound)):
                    for j in range(v_lowerbound, v_upperbound):
                        for k in range(z_lowerbound, z_upperbound):
                            sample.append(x[i - size: i + size + 1, j - size: j + size + 1, k - size: k + size + 1, :])
                            y_heat1.append(heat1[i - size][j - size][k - size])
                            y_heat2.append(heat2[i - size][j - size][k - size])
                            y_heat3.append(heat3[i - size][j - size][k - size])

                x_part = np.stack(sample, axis = 0)
                y_heat1 = np.array(y_heat1)
                y_heat2 = np.array(y_heat2)
                y_heat3 = np.array(y_heat3)

                del sample
                gc.collect()

                y_pred_12 = np.append(y_pred_12, loaded_model_1.predict(x_part).flatten())
                y_true_12 = np.append(y_true_12, y_heat1)

                y_pred_13 = np.append(y_pred_13, loaded_model_2.predict(x_part).flatten())
                y_true_13 = np.append(y_true_13, y_heat2)

                y_pred_23 = np.append(y_pred_23, loaded_model_3.predict(x_part).flatten())
                y_true_23 = np.append(y_true_23, y_heat3)

                del x_part
                del y_heat1
                del y_heat2
                del y_heat3
                gc.collect()

    out.append(['Re15sh20_DNScoarse' + str(coarse), 'Corr', str(np.round(np.corrcoef(y_pred_12, y_true_12)[0, 1], 4)), str(np.round(np.corrcoef(y_pred_13, y_true_13)[0, 1], 4)), str(np.round(np.corrcoef(y_pred_23, y_true_23)[0, 1], 4))])
    out.append(['', 'R^2', str(np.round(r2_score(y_true_12, y_pred_12), 4)), str(np.round(r2_score(y_true_13, y_pred_13), 4)), str(np.round(r2_score(y_true_23, y_pred_23), 4))])
    display(HTML(tabulate.tabulate(out, tablefmt = 'html')))

3 148
3 98
3 57



3 148
3 98
63 117



3 148
104 199
3 57



3 148
104 199
63 117



154 299
3 98
3 57


...


154 299
104 199
3 57



154 299
104 199
63 117


Data Source,Result,Heat1,Heat2,Heat3
Re15sh20_DNScoarse4,Corr,0.8206,0.5469,0.6654
,R^2,0.5564,0.2591,0.3065


3 74
3 49
3 30


Data Source,Result,Heat1,Heat2,Heat3
Re15sh20_DNScoarse4,Corr,0.8206,0.5469,0.6654
,R^2,0.5564,0.2591,0.3065
Re15sh20_DNScoarse16,Corr,0.7433,0.5964,0.7964
,R^2,0.4667,0.2346,0.5523
